# Clase de Pandas: Transformación, Filtrado y Operaciones con Columnas

En esta segunda clase de pandas, profundizaremos en:
- **Transformaciones**: cómo modificar y procesar datos de un DataFrame.
- **Filtrado** más avanzado (combinación de múltiples condiciones).
- **Operaciones con columnas**: crear nuevas columnas, eliminar columnas, renombrar, aplicar funciones de manera vectorizada, etc.

---


## 1. Introducción <a name="introduccion"></a>

Recordemos que **pandas** ofrece gran flexibilidad para preparar y transformar datos rápidamente. Veremos cómo realizar operaciones comunes, como:
- Crear nuevas columnas a partir de otras.
- Usar expresiones vectorizadas para transformar datos.
- Filtrar filas con condiciones más avanzadas.

---


In [2]:
# Importamos pandas y numpy (para algunos ejemplos)
import pandas as pd
import numpy as np

# Opcional: Ajustar algunas opciones de visualización
# pd.set_option('display.max_columns', None)  # Muestra todas las columnas en el output
# pd.set_option('display.width', 80)          # Ajusta el ancho para la consola


## 2. Preparación de Datos <a name="preparacion"></a>

Para ilustrar estas operaciones, creemos un DataFrame de ejemplo. (En un caso real, podríamos usar `pd.read_csv()` u otras funciones para cargar datos de un archivo.)

En este ejemplo, generaremos un DataFrame de ventas con las siguientes columnas:
- **Producto**
- **Precio Unitario**
- **Unidades Vendidas**
- **Descuento (%)**

---


In [3]:
    
datos_ventas = pd.read_csv('supermarket_sales - Sheet1.csv')

df = pd.DataFrame(datos_ventas)
df


,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,233-67-5758,C,Naypyitaw,Normal,Male,Health and beauty,40.35,1,2.0175,42.3675,1/29/2019,13:46,Ewallet,40.35,4.761905,2.0175,6.2
996,303-96-2227,B,Mandalay,Normal,Female,Home and lifestyle,97.38,10,48.6900,1022.4900,3/2/2019,17:16,Ewallet,973.80,4.761905,48.6900,4.4
997,727-02-1313,A,Yangon,Member,Male,Food and beverages,31.84,1,1.5920,33.4320,2/9/2019,13:22,Cash,31.84,4.761905,1.5920,7.7
998,347-56-2442,A,Yangon,Normal,Male,Home and lifestyle,65.82,1,3.2910,69.1110,2/22/2019,15:33,Cash,65.82,4.761905,3.2910,4.1


In [4]:
df.columns

Index(['Invoice ID', 'Branch', 'City', 'Customer type', 'Gender',
       'Product line', 'Unit price', 'Quantity', 'Tax 5%', 'Total', 'Date',
       'Time', 'Payment', 'cogs', 'gross margin percentage', 'gross income',
       'Rating'],
      dtype='object')

## 2.1 Renombrar solo la columna 'cogs'

Vamos a renombrar únicamente la columna `cogs` para que sea más clara: `Cost of Goods Sold`.


In [5]:
# Renombrar solo la columna 'cogs'
df.rename(columns={"cogs": "Cost of Goods Sold"}, inplace=True)

# Verificamos que se haya renombrado correctamente
print("Nombres de columnas después del renombrado:")
print(df.columns)


Nombres de columnas después del renombrado:
Index(['Invoice ID', 'Branch', 'City', 'Customer type', 'Gender',
       'Product line', 'Unit price', 'Quantity', 'Tax 5%', 'Total', 'Date',
       'Time', 'Payment', 'Cost of Goods Sold', 'gross margin percentage',
       'gross income', 'Rating'],
      dtype='object')


## 2.2 Crear nuevas columnas: Subtotal e Impuesto Calculado

Aunque el dataset ya trae columnas como 'Tax 5%' y 'Total', vamos a calcular el 'Subtotal' restando el impuesto, 
y comprobar si el impuesto realmente es el 5%.


In [6]:
# Subtotal = Total - Tax 5%
df["Subtotal"] = df["Total"] - df["Tax 5%"]

# Impuesto calculado (en porcentaje)
df["Impuesto Calculado (%)"] = (df["Tax 5%"] / df["Subtotal"]) * 100

# Mostramos una vista general
df[["Invoice ID", "Unit price", "Quantity", "Subtotal", "Tax 5%", "Total", "Impuesto Calculado (%)"]].head(10)


,Invoice ID,Unit price,Quantity,Subtotal,Tax 5%,Total,Impuesto Calculado (%)
0,750-67-8428,74.69,7,522.83,26.1415,548.9715,5.0
1,226-31-3081,15.28,5,76.40,3.8200,80.2200,5.0
2,631-41-3108,46.33,7,324.31,16.2155,340.5255,5.0
3,123-19-1176,58.22,8,465.76,23.2880,489.0480,5.0
4,373-73-7910,86.31,7,604.17,30.2085,634.3785,5.0
5,699-14-3026,85.39,7,597.73,29.8865,627.6165,5.0
6,355-53-5943,68.84,6,413.04,20.6520,433.6920,5.0
7,315-22-5665,73.56,10,735.60,36.7800,772.3800,5.0
8,665-32-9167,36.26,2,72.52,3.6260,76.1460,5.0
9,692-92-5582,54.84,3,164.52,8.2260,172.7460,5.0


## 2.3 Trabajando con fechas y horas

Convertimos las columnas de fecha y hora a tipos datetime para extraer información útil como año, mes, día, hora, etc.


In [7]:
df.head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,Cost of Goods Sold,gross margin percentage,gross income,Rating,Subtotal,Impuesto Calculado (%)
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1,522.83,5.0
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6,76.40,5.0
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4,324.31,5.0
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4,465.76,5.0
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3,604.17,5.0


In [8]:
df.head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,Cost of Goods Sold,gross margin percentage,gross income,Rating,Subtotal,Impuesto Calculado (%)
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1,522.83,5.0
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6,76.40,5.0
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4,324.31,5.0
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4,465.76,5.0
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3,604.17,5.0


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Invoice ID               1000 non-null   object 
 1   Branch                   1000 non-null   object 
 2   City                     1000 non-null   object 
 3   Customer type            1000 non-null   object 
 4   Gender                   1000 non-null   object 
 5   Product line             1000 non-null   object 
 6   Unit price               1000 non-null   float64
 7   Quantity                 1000 non-null   int64  
 8   Tax 5%                   1000 non-null   float64
 9   Total                    1000 non-null   float64
 10  Date                     1000 non-null   object 
 11  Time                     1000 non-null   object 
 12  Payment                  1000 non-null   object 
 13  Cost of Goods Sold       1000 non-null   float64
 14  gross margin percentage  

In [10]:
df_reducido = df[['Date','Time']].head()
df_reducido

,Date,Time
0,1/5/2019,13:08
1,3/8/2019,10:29
2,3/3/2019,13:23
3,1/27/2019,20:33
4,2/8/2019,10:37


In [11]:
'hola'+' Aaron'

'hola Aaron'

In [12]:

df_reducido['DateTime'] = df_reducido['Date']+ ' ' +df_reducido['Time']

In [13]:
df_reducido

,Date,Time,DateTime
0,1/5/2019,13:08,1/5/2019 13:08
1,3/8/2019,10:29,3/8/2019 10:29
2,3/3/2019,13:23,3/3/2019 13:23
3,1/27/2019,20:33,1/27/2019 20:33
4,2/8/2019,10:37,2/8/2019 10:37


In [14]:
df_reducido['DateTime'] = pd.to_datetime(df_reducido['DateTime'], format='%Y-%m-%d %H:%M')

ValueError: time data "1/5/2019 13:08" doesn't match format "%Y-%m-%d %H:%M", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [ ]:
df_reducido

,Date,Time,DateTime
0,1/5/2019,13:08,2019-01-05 13:08:00
1,3/8/2019,10:29,2019-03-08 10:29:00
2,3/3/2019,13:23,2019-03-03 13:23:00
3,1/27/2019,20:33,2019-01-27 20:33:00
4,2/8/2019,10:37,2019-02-08 10:37:00


In [ ]:
# Convertimos las columnas Fecha y Hora
df["Date"] = pd.to_datetime(df["Date"])
df["Time"] = pd.to_datetime(df["Time"], format="%H:%M")

# Extraemos partes útiles de fecha y hora
df["Año"] = df["Date"].dt.year
df["Mes"] = df["Date"].dt.month
df["Día de la Semana"] = df["Date"].dt.day_name()
df["Hora del Día"] = df["Time"].dt.hour

# Vista rápida
df[["Invoice ID", "Date", "Time", "Año", "Mes", "Día de la Semana", "Hora del Día"]]


,Invoice ID,Date,Time,Año,Mes,Día de la Semana,Hora del Día
0,750-67-8428,2019-01-05,1900-01-01 13:08:00,2019,1,Saturday,13
1,226-31-3081,2019-03-08,1900-01-01 10:29:00,2019,3,Friday,10
2,631-41-3108,2019-03-03,1900-01-01 13:23:00,2019,3,Sunday,13
3,123-19-1176,2019-01-27,1900-01-01 20:33:00,2019,1,Sunday,20
4,373-73-7910,2019-02-08,1900-01-01 10:37:00,2019,2,Friday,10
...,...,...,...,...,...,...,...
995,233-67-5758,2019-01-29,1900-01-01 13:46:00,2019,1,Tuesday,13
996,303-96-2227,2019-03-02,1900-01-01 17:16:00,2019,3,Saturday,17
997,727-02-1313,2019-02-09,1900-01-01 13:22:00,2019,2,Saturday,13
998,347-56-2442,2019-02-22,1900-01-01 15:33:00,2019,2,Friday,15


In [ ]:
df_reducido['DateTime'].dt.hour

0   2019-01-05 13:08:00
1   2019-03-08 10:29:00
2   2019-03-03 13:23:00
3   2019-01-27 20:33:00
4   2019-02-08 10:37:00
Name: DateTime, dtype: datetime64[ns]

In [ ]:
df_reducido[df_reducido['DateTime'] > '2019-01-01']

,Date,Time,DateTime
0,1/5/2019,13:08,2019-01-05 13:08:00
1,3/8/2019,10:29,2019-03-08 10:29:00
2,3/3/2019,13:23,2019-03-03 13:23:00
3,1/27/2019,20:33,2019-01-27 20:33:00
4,2/8/2019,10:37,2019-02-08 10:37:00


In [ ]:
pd.to_datetime('2020-03-05')

Timestamp('2020-03-05 00:00:00')

In [ ]:
pd.to_datetime(['2020-03-05','2020-03-06','2020-03-07']).month

Index([3, 3, 3], dtype='int32')

In [ ]:
pd.to_datetime(df_reducido['DateTime'])

0   2019-01-05 13:08:00
1   2019-03-08 10:29:00
2   2019-03-03 13:23:00
3   2019-01-27 20:33:00
4   2019-02-08 10:37:00
Name: DateTime, dtype: datetime64[ns]

In [ ]:
df_reducido['DateTime'].dt.month

0    1
1    3
2    3
3    1
4    2
Name: DateTime, dtype: int32

## 2.4 Clasificación con `apply`: Calificación (Rating)

Creamos una nueva columna que clasifica las calificaciones en:
- "Alta" si es mayor o igual a 9
- "Media" si está entre 7 y 9
- "Baja" si es menor a 7


In [ ]:
df[["Invoice ID", "Rating"]]

for r in df['Rating']:
    if r >= 9:
        print("Alta")
    elif r >= 7:
        print("Media")
    else:
        print("Baja")

Alta
Alta
Media
Media
Baja
Baja
Baja
Media
Media
Baja
Baja
Baja
Media
Media
Baja
Baja
Baja
Baja
Media
Baja
Baja
Baja
Baja
Alta
Baja
Media
Baja
Media
Alta
Media
Baja
Baja
Baja
Baja
Media
Baja
Media
Baja
Media
Media
Media
Baja
Baja
Media
Media
Alta
Media
Baja
Media
Media
Alta
Media
Baja
Baja
Baja
Media
Baja
Baja
Baja
Alta
Alta
Media
Alta
Media
Media
Baja
Baja
Alta
Baja
Baja
Alta
Baja
Baja
Media
Alta
Baja
Media
Baja
Baja
Baja
Baja
Baja
Baja
Media
Baja
Baja
Alta
Media
Baja
Baja
Media
Media
Baja
Media
Baja
Media
Baja
Baja
Alta
Media
Baja
Media
Baja
Alta
Media
Media
Media
Media
Baja
Alta
Baja
Media
Media
Baja
Alta
Media
Baja
Baja
Baja
Baja
Baja
Baja
Baja
Baja
Media
Baja
Alta
Baja
Media
Media
Media
Media
Media
Media
Baja
Baja
Media
Alta
Media
Alta
Baja
Media
Alta
Media
Media
Alta
Alta
Baja
Baja
Baja
Baja
Media
Media
Alta
Alta
Alta
Baja
Alta
Baja
Alta
Media
Baja
Media
Baja
Baja
Baja
Baja
Media
Baja
Media
Baja
Alta
Baja
Alta
Media
Media
Alta
Baja
Baja
Alta
Baja
Media
Media
Baja
Baja
Media
Media

In [ ]:
df['Invoice ID'].str.split('-').str[0]

0      [750, 67, 8428]
1      [226, 31, 3081]
2      [631, 41, 3108]
3      [123, 19, 1176]
4      [373, 73, 7910]
            ...       
995    [233, 67, 5758]
996    [303, 96, 2227]
997    [727, 02, 1313]
998    [347, 56, 2442]
999    [849, 09, 3807]
Name: Invoice ID, Length: 1000, dtype: object

In [ ]:
def clasificar_rating(r):
    if r >= 9:
        return "Alta"
    elif r >= 7:
        return "Media"
    else:
        return "Baja"

df["Nivel de Calificación"] = df["Rating"].apply(clasificar_rating)

df[["Invoice ID", "Rating", "Nivel de Calificación"]]


,Invoice ID,Rating,Nivel de Calificación
0,750-67-8428,9.1,Alta
1,226-31-3081,9.6,Alta
2,631-41-3108,7.4,Media
3,123-19-1176,8.4,Media
4,373-73-7910,5.3,Baja
...,...,...,...
995,233-67-5758,6.2,Baja
996,303-96-2227,4.4,Baja
997,727-02-1313,7.7,Media
998,347-56-2442,4.1,Baja


## 🧼 Limpieza de datos: duplicados y valores nulos

En esta sección aprenderemos a usar algunos de los métodos más comunes de pandas para **limpiar un DataFrame**:

---

### 🔁 Duplicados

- `duplicated()`  
Devuelve una serie booleana que indica si una fila está duplicada respecto a una anterior.

- `drop_duplicates()`  
Elimina filas duplicadas. Se puede usar sobre todas las columnas o sobre un subconjunto específico.



In [15]:
df

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,Cost of Goods Sold,gross margin percentage,gross income,Rating,Subtotal,Impuesto Calculado (%)
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1,522.83,5.0
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6,76.40,5.0
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4,324.31,5.0
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4,465.76,5.0
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3,604.17,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,233-67-5758,C,Naypyitaw,Normal,Male,Health and beauty,40.35,1,2.0175,42.3675,1/29/2019,13:46,Ewallet,40.35,4.761905,2.0175,6.2,40.35,5.0
996,303-96-2227,B,Mandalay,Normal,Female,Home and lifestyle,97.38,10,48.6900,1022.4900,3/2/2019,17:16,Ewallet,973.80,4.761905,48.6900,4.4,973.80,5.0
997,727-02-1313,A,Yangon,Member,Male,Food and beverages,31.84,1,1.5920,33.4320,2/9/2019,13:22,Cash,31.84,4.761905,1.5920,7.7,31.84,5.0
998,347-56-2442,A,Yangon,Normal,Male,Home and lifestyle,65.82,1,3.2910,69.1110,2/22/2019,15:33,Cash,65.82,4.761905,3.2910,4.1,65.82,5.0


In [16]:
df.columns

Index(['Invoice ID', 'Branch', 'City', 'Customer type', 'Gender',
       'Product line', 'Unit price', 'Quantity', 'Tax 5%', 'Total', 'Date',
       'Time', 'Payment', 'Cost of Goods Sold', 'gross margin percentage',
       'gross income', 'Rating', 'Subtotal', 'Impuesto Calculado (%)'],
      dtype='object')

### Acotamos el df

In [17]:
df_short = df[['Invoice ID', 'Gender','Total','Rating', 'City']] 
df_short.head()

,Invoice ID,Gender,Total,Rating,City
0,750-67-8428,Female,548.9715,9.1,Yangon
1,226-31-3081,Female,80.2200,9.6,Naypyitaw
2,631-41-3108,Male,340.5255,7.4,Yangon
3,123-19-1176,Male,489.0480,8.4,Yangon
4,373-73-7910,Male,634.3785,5.3,Yangon


In [27]:
df_short

,Invoice ID,Gender,Total,Rating,City
0,750-67-8428,Female,548.9715,9.1,Yangon
1,226-31-3081,Female,80.2200,9.6,Naypyitaw
2,631-41-3108,Male,340.5255,7.4,Yangon
3,123-19-1176,Male,489.0480,8.4,Yangon
4,373-73-7910,Male,634.3785,5.3,Yangon
...,...,...,...,...,...
995,233-67-5758,Male,42.3675,6.2,Naypyitaw
996,303-96-2227,Female,1022.4900,4.4,Mandalay
997,727-02-1313,Male,33.4320,7.7,Yangon
998,347-56-2442,Male,69.1110,4.1,Yangon


In [37]:

df_with_dup = pd.concat([df_short.iloc[:500,:],df_short],axis=0)

In [46]:
df_with_dup.loc[0,'Gender']

0    Female
0    Female
Name: Gender, dtype: object

### Reinicia el índice para hacerlo único

In [50]:
df_with_dup.reset_index(drop=True, inplace=True)

In [51]:
df_with_dup

,Invoice ID,Gender,Total,Rating,City
0,750-67-8428,Female,548.9715,9.1,Yangon
1,226-31-3081,Female,80.2200,9.6,Naypyitaw
2,631-41-3108,Male,340.5255,7.4,Yangon
3,123-19-1176,Male,489.0480,8.4,Yangon
4,373-73-7910,Male,634.3785,5.3,Yangon
...,...,...,...,...,...
1495,233-67-5758,Male,42.3675,6.2,Naypyitaw
1496,303-96-2227,Female,1022.4900,4.4,Mandalay
1497,727-02-1313,Male,33.4320,7.7,Yangon
1498,347-56-2442,Male,69.1110,4.1,Yangon


#### Ver los duplicados de mi df

In [33]:
df_with_dup[df_with_dup.duplicated()]

,Invoice ID,Gender,Total,Rating,City
0,750-67-8428,Female,548.9715,9.1,Yangon
1,226-31-3081,Female,80.2200,9.6,Naypyitaw
2,631-41-3108,Male,340.5255,7.4,Yangon
3,123-19-1176,Male,489.0480,8.4,Yangon
4,373-73-7910,Male,634.3785,5.3,Yangon
...,...,...,...,...,...
495,632-32-4574,Male,637.7280,5.5,Mandalay
496,556-97-7101,Female,132.7620,8.5,Naypyitaw
497,862-59-8517,Female,568.5120,6.2,Naypyitaw
498,401-18-8016,Female,103.0365,8.9,Mandalay


In [ ]:


df_with_dup = pd.concat([df_short.iloc[:500,:],df_short],axis=0)

filas_nulas = pd.DataFrame([
    {"Invoice ID": None, "Gender": None, "Total": np.nan, "Rating": np.nan}, 
    {"Invoice ID": "000-00-0000", "Gender": "Female", "Total": 300.0, "Rating": np.nan},
    {"Invoice ID": "111-11-1111", "Gender": None, "Total": None, "Rating": 9.0}
])

# pandas rellenará con NaN las columnas que falten al unir con df
df_with_dup_and_na = pd.concat([df_with_dup, filas_nulas], axis=0, ignore_index=True)

In [55]:
filas_nulas

,Invoice ID,Gender,Total,Rating
0,None,None,NaN,NaN
1,000-00-0000,Female,300.0,NaN
2,111-11-1111,None,NaN,9.0


In [54]:
df_with_dup_and_na

,Invoice ID,Gender,Total,Rating,City
0,750-67-8428,Female,548.9715,9.1,Yangon
1,226-31-3081,Female,80.2200,9.6,Naypyitaw
2,631-41-3108,Male,340.5255,7.4,Yangon
3,123-19-1176,Male,489.0480,8.4,Yangon
4,373-73-7910,Male,634.3785,5.3,Yangon
...,...,...,...,...,...
1498,347-56-2442,Male,69.1110,4.1,Yangon
1499,849-09-3807,Female,649.2990,6.6,Yangon
1500,None,None,NaN,NaN,NaN
1501,000-00-0000,Female,300.0000,NaN,NaN


In [56]:
# 🔁 Verificar si hay filas duplicadas
# True indica que esa fila ya ha aparecido antes con los mismos valores
duplicados = df_with_dup_and_na.duplicated()
print("¿Hay filas duplicadas?")
print(duplicados.value_counts())
# print(df['City'].duplicated())


¿Hay filas duplicadas?
False    1003
True      500
Name: count, dtype: int64


### value_counts

In [58]:
df_with_dup_and_na['Gender'].value_counts()

Gender
Male      753
Female    748
Name: count, dtype: int64

In [138]:
df_with_dup_and_na[df_with_dup_and_na.duplicated()==True]

,Invoice ID,Gender,Total,Rating,City
500,750-67-8428,Female,548.9715,9.1,Yangon
501,226-31-3081,Female,80.2200,9.6,Naypyitaw
502,631-41-3108,Male,340.5255,7.4,Yangon
503,123-19-1176,Male,489.0480,8.4,Yangon
504,373-73-7910,Male,634.3785,5.3,Yangon
...,...,...,...,...,...
995,632-32-4574,Male,637.7280,5.5,Mandalay
996,556-97-7101,Female,132.7620,8.5,Naypyitaw
997,862-59-8517,Female,568.5120,6.2,Naypyitaw
998,401-18-8016,Female,103.0365,8.9,Mandalay


In [59]:
# 🔁 Eliminar duplicados
# Borra las filas duplicadas y devuelve un nuevo DataFrame (no modifica el original a menos que uses inplace=True)
df_sin_duplicados = df_with_dup_and_na.drop_duplicates()
print(f"Número de filas después de eliminar duplicados: {len(df_sin_duplicados)}")


Número de filas después de eliminar duplicados: 1003


In [ ]:
df_with_dup_and_na.drop_duplicates(subset=)

,Invoice ID,Gender,Total,Rating,City
0,750-67-8428,Female,548.9715,9.1,Yangon
1,226-31-3081,Female,80.2200,9.6,Naypyitaw
2,631-41-3108,Male,340.5255,7.4,Yangon
3,123-19-1176,Male,489.0480,8.4,Yangon
4,373-73-7910,Male,634.3785,5.3,Yangon
...,...,...,...,...,...
1498,347-56-2442,Male,69.1110,4.1,Yangon
1499,849-09-3807,Female,649.2990,6.6,Yangon
1500,None,None,NaN,NaN,NaN
1501,000-00-0000,Female,300.0000,NaN,NaN


In [140]:
df_with_dup_and_na.drop_duplicates(subset=['Invoice ID','Total'])

,Invoice ID,Gender,Total,Rating,City
0,750-67-8428,Female,548.9715,9.1,Yangon
1,226-31-3081,Female,80.2200,9.6,Naypyitaw
2,631-41-3108,Male,340.5255,7.4,Yangon
3,123-19-1176,Male,489.0480,8.4,Yangon
4,373-73-7910,Male,634.3785,5.3,Yangon
...,...,...,...,...,...
1498,347-56-2442,Male,69.1110,4.1,Yangon
1499,849-09-3807,Female,649.2990,6.6,Yangon
1500,None,None,NaN,NaN,NaN
1501,000-00-0000,Female,300.0000,NaN,NaN


---

### ❓ Valores nulos

- `isna()` o `isnull()`  
Devuelve `True` donde hay valores nulos en el DataFrame.

- `dropna()`  
Elimina filas o columnas que contienen valores nulos.

- `fillna(valor)`  
Permite rellenar los valores nulos con un valor específico (por ejemplo, 0 o la media de una columna).

---

Estos métodos son fundamentales para preparar los datos antes de analizarlos o visualizarlos.


In [69]:
# ❓ Buscar valores nulos en todo el DataFrame
nulos = df_with_dup_and_na.isna().sum()
print("Valores nulos por columna:")
print(nulos)


Valores nulos por columna:
Invoice ID    1
Gender        2
Total         2
Rating        2
City          3
dtype: int64


In [70]:
# ❓ Eliminar filas con cualquier valor nulo
# ⚠️ Cuidado: esta operación puede eliminar muchas filas si no se usa con criterio
df_sin_nulos = df_with_dup_and_na.dropna()
print(f"Número de filas tras eliminar nulos: {len(df_sin_nulos)}")


Número de filas tras eliminar nulos: 1500


In [82]:
df_with_dup_and_na.dropna(subset='Gender',ignore_index=True)

,Invoice ID,Gender,Total,Rating,City
0,750-67-8428,Female,548.9715,9.1,Yangon
1,226-31-3081,Female,80.2200,9.6,Naypyitaw
2,631-41-3108,Male,340.5255,7.4,Yangon
3,123-19-1176,Male,489.0480,8.4,Yangon
4,373-73-7910,Male,634.3785,5.3,Yangon
...,...,...,...,...,...
1496,303-96-2227,Female,1022.4900,4.4,Mandalay
1497,727-02-1313,Male,33.4320,7.7,Yangon
1498,347-56-2442,Male,69.1110,4.1,Yangon
1499,849-09-3807,Female,649.2990,6.6,Yangon


In [87]:
df_with_dup_and_na[(df_with_dup_and_na['City'].notna()) & 
                   (df_with_dup_and_na['Rating'].notna())]

,Invoice ID,Gender,Total,Rating,City
0,750-67-8428,Female,548.9715,9.1,Yangon
1,226-31-3081,Female,80.2200,9.6,Naypyitaw
2,631-41-3108,Male,340.5255,7.4,Yangon
3,123-19-1176,Male,489.0480,8.4,Yangon
4,373-73-7910,Male,634.3785,5.3,Yangon
...,...,...,...,...,...
1495,233-67-5758,Male,42.3675,6.2,Naypyitaw
1496,303-96-2227,Female,1022.4900,4.4,Mandalay
1497,727-02-1313,Male,33.4320,7.7,Yangon
1498,347-56-2442,Male,69.1110,4.1,Yangon


In [90]:
# ❓ Rellenar valores nulos con un valor específico
# Ejemplo: rellenamos nulos en "Rating" con la media
media_rating = df_with_dup_and_na["Rating"].mean()
df_with_dup_and_na["Rating"] =df_with_dup_and_na["Rating"].fillna(media_rating)



print("Comprobamos si quedan nulos en Rating:")
print(df_with_dup_and_na["Rating"].isna().sum())


Comprobamos si quedan nulos en Rating:
0


In [89]:
media_rating

np.float64(6.989340439706862)

In [49]:
df_with_dup_and_na

,Invoice ID,Gender,Total,Rating,City
0,750-67-8428,Female,548.9715,9.10000,Yangon
1,226-31-3081,Female,80.2200,9.60000,Naypyitaw
2,631-41-3108,Male,340.5255,7.40000,Yangon
3,123-19-1176,Male,489.0480,8.40000,Yangon
4,373-73-7910,Male,634.3785,5.30000,Yangon
...,...,...,...,...,...
1498,347-56-2442,Male,69.1110,4.10000,Yangon
1499,849-09-3807,Female,649.2990,6.60000,Yangon
1500,None,None,NaN,6.98934,NaN
1501,000-00-0000,Female,300.0000,6.98934,NaN


In [96]:
df.groupby('City')['Total'].sum()

City
Mandalay     106197.6720
Naypyitaw    110568.7065
Yangon       106200.3705
Name: Total, dtype: float64

In [98]:
df_short.columns

Index(['Invoice ID', 'Gender', 'Total', 'Rating', 'City'], dtype='object')